In [1]:
import os
import shutil
import tensorflow
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from PIL import Image

MUSHROOMS_PATH = 'mushrooms_dataset'

# Directory for the images and its subdirectories
images_dir = os.path.join(MUSHROOMS_PATH, 'images')
subdirs = [os.path.join(images_dir, subdir) for subdir in os.listdir(images_dir) if os.path.isdir(os.path.join(images_dir, subdir))]

In [2]:
#Now we have some ideas for dividing the dataset into training and testing sets. We can use the train_test_split function from scikit-learn to divide the dataset into training and testing sets.
#But for that we will have to put the every image into array and then into a dataframe
#Then we will have to use ImageDataGenerator and flow_from_dataframe to load the images from the dataframe

#Second idea is to manually create the test set by taking 20% of the images from each class and putting them into a separate directory.
#We will then use ImageDataGenerator and flow_from_directory to load the images from the directory.

#In both ideas we need to take in consider stratification, so that the distribution of classes in the training and testing sets is similar.
#For example, if in one class there are 10 images and in another one there are 8 images, we want both  of them to have the same percentage of images in the training and testing sets.

#Third idea is to use the splitfolders library to divide the dataset into training and testing sets.
#But again we have to stratify the dataset which is not supported by that library.

#So the first idea might require a lot of memory usage, the second idea needs us to well do this manually which is not very efficient.
#And the third idea is not supporting stratification.

#So for now we will use the first idea and divide the dataset into training and testing sets using the train_test_split function from scikit-learn which has the stratify parameter.


In [3]:
#So the process with the first idea is as follows:
#1. Load the images and its corresponding labels into a dataframe.
#2. Divide the dataset into training and testing sets using the train_test_split function from scikit-learn with stratification.
#3. Use ImageDataGenerator and flow_from_dataframe to load the images from the dataframe.

In [4]:
data = []
for subdir in subdirs:
    label = os.path.basename(subdir) #we specify the label for each image
    for filename in os.listdir(subdir):
        if filename.endswith('.jpg'):
            data.append((os.path.join(subdir, filename), label)) #we need to include whole path of the image for using flow_from_dataframe because it reads the images directly from the file system using the paths provided in the DataFrame.
data_df = pd.DataFrame(data, columns=['filename', 'label'])

In [5]:
data_df.head()

,filename,label
0,mushrooms_dataset\images\Abortiporus_biennis\1...,Abortiporus_biennis
1,mushrooms_dataset\images\Abortiporus_biennis\1...,Abortiporus_biennis
2,mushrooms_dataset\images\Abortiporus_biennis\1...,Abortiporus_biennis
3,mushrooms_dataset\images\Abortiporus_biennis\1...,Abortiporus_biennis
4,mushrooms_dataset\images\Abortiporus_biennis\1...,Abortiporus_biennis


In [6]:
train_df, test_df = train_test_split(data_df, test_size=0.2, stratify=data_df['label'], random_state=42)

In [7]:
datagen = ImageDataGenerator(rescale=1./255, zoom_range=0.2, shear_range=0.2, validation_split=0.25,) #we use 25% from the 80% of the training set as the validation set which will be the same amount as the testing set

train_data = datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='filename',
    y_col='label',
    target_size=(128, 128),
    class_mode='categorical',
    batch_size=64,
    subset='training'
)


val_data = datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='filename',
    y_col='label',
    target_size=(128, 128),
    class_mode='categorical',
    batch_size=64,
    subset='validation'
)

datagen_test = ImageDataGenerator(rescale=1./255)

test_data = datagen_test.flow_from_dataframe(
    dataframe=test_df,
    x_col='filename',
    y_col='label',
    target_size=(128, 128),
    class_mode='categorical',
    batch_size=64
)

Found 115491 validated image filenames belonging to 1727 classes.
Found 38497 validated image filenames belonging to 1727 classes.
Found 38498 validated image filenames belonging to 1727 classes.


In [8]:
#---DONE---
#It worked, but why do we have only 6903 classes in the test set and 7504 in training and validation sets? 
# Perhaps there are not enough images in some classes???

# It is probably true beacause when we use stratify parameter in train_test_split function, it tries to keep the distribution of classes in the training and testing sets similar.
# But if there are not enough images in some classes, it will not be able to keep the distribution of classes similar in the training and testing sets.
# So we have few solutions to this
# 1. Ensure that each class has a minimum number of instances before splitting the data into training and testing sets - that worked!!!!
# 2. Use the stratify sampling only on the classes with sufficient instances, and randomly split the ones with too few instances

Could have - we could create our own model but since we have massive amount of images its easier to use pre-trained one

In [9]:
# Creating the first model
# model = Sequential()
# model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
# model.add(MaxPooling2D((2, 2)))
# model.add(Conv2D(64, (3, 3), activation='relu'))
# model.add(MaxPooling2D((2, 2)))
# model.add(Conv2D(128, (3, 3), activation='relu'))
# model.add(MaxPooling2D((2, 2)))
# model.add(Flatten())
# model.add(Dense(128, activation='relu'))
# model.add(Dense(7504, activation='softmax'))

# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# history = model.fit(train_data, validation_data=val_data, epochs=20)



In [10]:
# model.evaluate(test_data)

Since we will use couple of models to compare each ones results, its good to create a function for saving model

In [18]:
def saveModel(model, model_name):

    if not os.path.exists('models'):
        os.makedirs('models')
    model.save(f'models/{model_name}.h5')

Training on ResNet50 model

In [12]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras import Model

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)

predictions = Dense(1727, activation='softmax')(x)

resnet_model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

resnet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
resnet_model.fit(train_data, validation_data=val_data, epochs=10)





Epoch 1/10


1805/1805 [==============================] - 868s 480ms/step - loss: 7.0766 - accuracy: 0.0139 - val_loss: 7.0082 - val_accuracy: 0.0140
Epoch 2/10
1805/1805 [==============================] - 755s 418ms/step - loss: 7.0092 - accuracy: 0.0143 - val_loss: 6.9702 - val_accuracy: 0.0142
Epoch 3/10
1805/1805 [==============================] - 759s 421ms/step - loss: 6.9797 - accuracy: 0.0145 - val_loss: 6.9394 - val_accuracy: 0.0147
Epoch 4/10
1805/1805 [==============================] - 756s 419ms/step - loss: 6.9510 - accuracy: 0.0149 - val_loss: 6.9109 - val_accuracy: 0.0151
Epoch 5/10
1805/1805 [==============================] - 769s 426ms/step - loss: 6.9203 - accuracy: 0.0152 - val_loss: 6.8801 - val_accuracy: 0.0161
Epoch 6/10
1805/1805 [==============================] - 773s 428ms/step - loss: 6.8866 - accuracy: 0.0163 - val_loss: 6.8447 - val_accuracy: 0.0167
Epoch 7/10
1805/1805 [==============================] - 746s 413ms/step - loss: 6.8588 - accuracy: 0.0162 -

In [13]:
# lets check how many layers model has
for i, layer in enumerate(resnet_model.layers):
    print(i, layer.name)

0 input_1
1 conv1_pad
2 conv1_conv
3 conv1_bn
4 conv1_relu
5 pool1_pad
6 pool1_pool
7 conv2_block1_1_conv
8 conv2_block1_1_bn
9 conv2_block1_1_relu
10 conv2_block1_2_conv
11 conv2_block1_2_bn
12 conv2_block1_2_relu
13 conv2_block1_0_conv
14 conv2_block1_3_conv
15 conv2_block1_0_bn
16 conv2_block1_3_bn
17 conv2_block1_add
18 conv2_block1_out
19 conv2_block2_1_conv
20 conv2_block2_1_bn
21 conv2_block2_1_relu
22 conv2_block2_2_conv
23 conv2_block2_2_bn
24 conv2_block2_2_relu
25 conv2_block2_3_conv
26 conv2_block2_3_bn
27 conv2_block2_add
28 conv2_block2_out
29 conv2_block3_1_conv
30 conv2_block3_1_bn
31 conv2_block3_1_relu
32 conv2_block3_2_conv
33 conv2_block3_2_bn
34 conv2_block3_2_relu
35 conv2_block3_3_conv
36 conv2_block3_3_bn
37 conv2_block3_add
38 conv2_block3_out
39 conv3_block1_1_conv
40 conv3_block1_1_bn
41 conv3_block1_1_relu
42 conv3_block1_2_conv
43 conv3_block1_2_bn
44 conv3_block1_2_relu
45 conv3_block1_0_conv
46 conv3_block1_3_conv
47 conv3_block1_0_bn
48 conv3_block1_3_bn

In [14]:
# lets unfreeze some layers
for layer in resnet_model.layers[:143]:
    layer.trainable = False
for layer in resnet_model.layers[143:]:
    layer.trainable = True

In [15]:
# lets compile the model again and train it
resnet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
resnet_model.fit(train_data, validation_data=val_data, epochs=30)

Epoch 1/30
1805/1805 [==============================] - 859s 474ms/step - loss: 7.1371 - accuracy: 0.0139 - val_loss: 7.0354 - val_accuracy: 0.0140
Epoch 2/30
1805/1805 [==============================] - 854s 473ms/step - loss: 7.0205 - accuracy: 0.0141 - val_loss: 6.9905 - val_accuracy: 0.0140
Epoch 3/30
1805/1805 [==============================] - 853s 472ms/step - loss: 6.9775 - accuracy: 0.0147 - val_loss: 7.0051 - val_accuracy: 0.0141
Epoch 4/30
1805/1805 [==============================] - 853s 472ms/step - loss: 6.9375 - accuracy: 0.0147 - val_loss: 6.8974 - val_accuracy: 0.0150
Epoch 5/30
1805/1805 [==============================] - 854s 473ms/step - loss: 6.8952 - accuracy: 0.0154 - val_loss: 6.8329 - val_accuracy: 0.0176
Epoch 6/30
1805/1805 [==============================] - 856s 474ms/step - loss: 6.8244 - accuracy: 0.0171 - val_loss: 6.8324 - val_accuracy: 0.0184
Epoch 7/30
1805/1805 [==============================] - 855s 473ms/step - loss: 6.7849 - accuracy: 0.0171 - val_

In [19]:
# lets evaluate the model and save it
resnet_model.evaluate(test_data)
saveModel(resnet_model, 'resnet_model')

602/602 [==============================] - 160s 266ms/step - loss: 6.4663 - accuracy: 0.0264


C:\Users\Adam\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Training on InceptionV3 model

In [20]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)

x = Dense(1024, activation='relu')(x)
predictions = Dense(1727, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_data, validation_data=val_data, epochs=10)

Epoch 1/10
1805/1805 [==============================] - 536s 295ms/step - loss: 6.5079 - accuracy: 0.0300 - val_loss: 6.1377 - val_accuracy: 0.0407
Epoch 2/10
1805/1805 [==============================] - 502s 278ms/step - loss: 5.9192 - accuracy: 0.0467 - val_loss: 5.9074 - val_accuracy: 0.0468
Epoch 3/10
1805/1805 [==============================] - 501s 277ms/step - loss: 5.7388 - accuracy: 0.0547 - val_loss: 5.8419 - val_accuracy: 0.0520
Epoch 4/10
1805/1805 [==============================] - 489s 271ms/step - loss: 5.6420 - accuracy: 0.0594 - val_loss: 5.7502 - val_accuracy: 0.0567
Epoch 5/10
1805/1805 [==============================] - 487s 270ms/step - loss: 5.5804 - accuracy: 0.0629 - val_loss: 5.7331 - val_accuracy: 0.0601
Epoch 6/10
1805/1805 [==============================] - 483s 268ms/step - loss: 5.5447 - accuracy: 0.0654 - val_loss: 5.7264 - val_accuracy: 0.0586
Epoch 7/10
1805/1805 [==============================] - 482s 267ms/step - loss: 5.5236 - accuracy: 0.0662 - val_

In [21]:
for i, layer in enumerate(base_model.layers):
    print(i, layer.name)

0 input_2
1 conv2d
2 batch_normalization
3 activation
4 conv2d_1
5 batch_normalization_1
6 activation_1
7 conv2d_2
8 batch_normalization_2
9 activation_2
10 max_pooling2d
11 conv2d_3
12 batch_normalization_3
13 activation_3
14 conv2d_4
15 batch_normalization_4
16 activation_4
17 max_pooling2d_1
18 conv2d_8
19 batch_normalization_8
20 activation_8
21 conv2d_6
22 conv2d_9
23 batch_normalization_6
24 batch_normalization_9
25 activation_6
26 activation_9
27 average_pooling2d
28 conv2d_5
29 conv2d_7
30 conv2d_10
31 conv2d_11
32 batch_normalization_5
33 batch_normalization_7
34 batch_normalization_10
35 batch_normalization_11
36 activation_5
37 activation_7
38 activation_10
39 activation_11
40 mixed0
41 conv2d_15
42 batch_normalization_15
43 activation_15
44 conv2d_13
45 conv2d_16
46 batch_normalization_13
47 batch_normalization_16
48 activation_13
49 activation_16
50 average_pooling2d_1
51 conv2d_12
52 conv2d_14
53 conv2d_17
54 conv2d_18
55 batch_normalization_12
56 batch_normalization_14
5

In [22]:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

In [23]:
from tensorflow.keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

In [24]:
model.fit(train_data, validation_data=val_data, epochs=30)

Epoch 1/30
1805/1805 [==============================] - 590s 325ms/step - loss: 5.2877 - accuracy: 0.0779 - val_loss: 5.1379 - val_accuracy: 0.0984
Epoch 2/30
1805/1805 [==============================] - 585s 324ms/step - loss: 4.8073 - accuracy: 0.1184 - val_loss: 5.0015 - val_accuracy: 0.1152
Epoch 3/30
1805/1805 [==============================] - 585s 324ms/step - loss: 4.5421 - accuracy: 0.1458 - val_loss: 4.8162 - val_accuracy: 0.1347
Epoch 4/30
1805/1805 [==============================] - 589s 326ms/step - loss: 4.3195 - accuracy: 0.1711 - val_loss: 4.7508 - val_accuracy: 0.1416
Epoch 5/30
1805/1805 [==============================] - 588s 326ms/step - loss: 4.1139 - accuracy: 0.1958 - val_loss: 4.7058 - val_accuracy: 0.1513
Epoch 6/30
1805/1805 [==============================] - 589s 326ms/step - loss: 3.9242 - accuracy: 0.2189 - val_loss: 4.6601 - val_accuracy: 0.1619
Epoch 7/30
1805/1805 [==============================] - 587s 325ms/step - loss: 3.7418 - accuracy: 0.2448 - val_

In [25]:
#Evaluate the model
model.evaluate(test_data)
#Save the model
saveModel(model, 'inception_v3_mushrooms')

602/602 [==============================] - 79s 130ms/step - loss: 5.8689 - accuracy: 0.2356
